In [1]:
# from structures import Block, Disk
from utils import *

In [2]:
# data = parse_data()
# sort by averageRating
# data.sort(key=lambda record: record[1]) 

In [3]:
# for record in data:
#     tconst, average_rating, num_votes = record
#     tconst_bytes = convert_string_to_bytes(tconst, 10)
#     average_rating_bytes = convert_float_to_bytes(average_rating)
#     num_votes_bytes = convert_uint_to_bytes(num_votes)
#     assert(convert_bytes_to_string(tconst_bytes) == tconst)
#     assert(convert_bytes_to_float(average_rating_bytes) == average_rating)
#     assert(convert_bytes_to_uint(num_votes_bytes) == num_votes)

In [4]:
# test_block = Block()
# set_data_block_header(test_block, 7)
# get_data_block_header(test_block)
# get_block_type(test_block)

In [5]:
# test_block = Block()
# set_index_block_header(test_block, 23, 22, "root")
# get_index_block_header(test_block)
# get_block_type(test_block)

In [6]:
# test_block = Block()
# set_index_block_header(test_block, 23, 22, "leaf")
# get_index_block_header(test_block)
# get_block_type(test_block)

In [7]:
# data[:2]

In [8]:
# test_block = Block()
# set_data_block_header(test_block, 3)
# insert_record_bytes(test_block, convert_record_to_bytes(data[0]))
# insert_record_bytes(test_block, convert_record_to_bytes(data[1]))

In [9]:
# print(convert_bytes_to_record(test_block.bytes[9:27]))
# print(convert_bytes_to_record(test_block.bytes[27:27+18]))

In [10]:
# while insert_record_bytes(test_block, convert_record_to_bytes(data[0])):
#     pass

In [11]:
# for i in range(9, 100, 18):
#     if i + 18 <= 100:
#         print(convert_bytes_to_record(read_record_bytes(test_block, i)))

In [12]:
# block = Block()

In [13]:
# set_index_block_header(block, 0, 0, "root")

In [14]:
# get_index_block_header(block)

In [15]:
# pointers, keys = deserialize_index_block(block)

In [16]:
# pointers, keys

In [17]:
# pointers.append((3, 100))
# keys.append(5)

In [18]:
# serialized = serialize_index_block(pointers, keys)
# print(len(serialized))

In [19]:
# set_key_pointers_bytes(block, serialized)

In [20]:
# serialize_index_block(pointers, keys)

In [21]:
# print(block)

In [22]:
# deserialize_index_block(block)

In [23]:
# to deal with duplicate key -> record header need to have pointer (block_and_offset) of next record with same average_rating

In [37]:
class BPTree:
    
    def __init__(self):
        self.root = BPTreeNode(True, True)

    def insert(self, key, value):
        self.root = self.root.insert(key, value)
        
    def search(self, key):
        return self.root.search(key)
    
    def search_range(self, lower, upper):
        return self.root.search_range(lower, upper)
    
    def delete(self, key):
#         if not self.search(key):
#             return
        self.root.delete(key, None, None, None, float("inf"), float("-inf"))
        if self.root.keys == []:
            self.root = self.root.pointers[0]
            if self.root:
                self.root.root = True
            
    def validate(self):
        if self.root:
            self.root.validate()
        
    def show(self):
        # bfs
        cur = [self.root]
        while cur:
            nxt = []
            to_print = []
            for node in cur:
                if node == None:
                    print("Empty tree")
                    return
                for key in node.keys:
                    to_print.append(key)
                to_print.append("|")
                for pointer in node.pointers:
                    if type(pointer) != BPTreeNode:
                        break
                    nxt.append(pointer)
            print(" ".join(str(x) for x in to_print))
            print()
            cur = nxt
        

In [74]:
class BPTreeNode:
    
    def __init__(self, leaf=False, root=False):
        self.keys = []
        self.pointers = [None] # each element is either List[val] or BPTreeNode
        self.leaf = leaf # boolean to denote whether node is leaf or not
        self.root = root # boolean to denote whether node is root or not
        self.max_keys = 6
        self.min_leaf_keys = (self.max_keys + 1) // 2
        self.min_non_leaf_keys = self.max_keys // 2
        # minimum value in subtree rooted at self (only used for deletion, the value is irrelevant in other operations)
        self.min_val = float("inf")
        
    def insert(self, key, value):
        """
        Inserts key-value into the tree
        If key does not exist, create key: [value], else append to existing list of values
        Returns the root of the tree (may be changed if there is overflow on the node where the key is inserted)
        """
        if self.leaf:
            inserted = False
            for i in range(len(self.keys)):
                if key < self.keys[i]:
                    self.keys.insert(i, key)
                    self.pointers.insert(i, [value])
                    inserted = True
                    break
                elif key == self.keys[i]:
                    self.pointers[i].append(value)
                    inserted = True
                    break
            if not inserted:
                pos = len(self.keys)
                self.keys.insert(pos, key)
                self.pointers.insert(pos, [value])
            # check if exceeds the maximum number of keys
            if len(self.keys) > self.max_keys:
                num_left = (len(self.keys) + 1) // 2 # number of keys allocated to left side
                
                right_node = BPTreeNode(leaf=True, root=False)
                right_node.keys = self.keys[num_left:]
                right_node.pointers = self.pointers[num_left:]
                
                self.keys = self.keys[:num_left]
                self.pointers = self.pointers[:num_left]
                self.pointers.append(right_node) # pointer to the immediate right leaf node (for range queries)
                
                # above node will either be the new parent (if root is leaf), or merged with existing parent
                above_node = BPTreeNode(leaf=False, root=True)
                above_node.keys = [right_node.keys[0]]
                above_node.pointers = [self, right_node]
                return above_node
            # if no underflow, return self. This is used by caller to check if there was an overflow
            return self
        
        else:
            # pointers[pos] is the subtree to recursively call on
            pos = None
            for i in range(len(self.keys)):
                if key < self.keys[i]:
                    pos = i
                    break
            if pos == None:
                pos = len(self.keys)
            to_merge = self.pointers[pos].insert(key, value)
            if to_merge is self.pointers[pos]:
                # since we know there is no overflow, we are done
                return self
            # to_merge is the node to be merged with self (it is actually above_node)
            self.pointers[pos] = to_merge.pointers[1]
            self.pointers.insert(pos, to_merge.pointers[0])
            self.keys.insert(pos, to_merge.keys[0])
            
            # non leaf can overflow as well
            # refer to slide 22 of lecture 8
            if len(self.keys) > self.max_keys:
                num_left = len(self.keys) // 2
                
                right_node = BPTreeNode(leaf=False, root=False)
                right_node.keys = self.keys[num_left+1:]
                right_node.pointers = self.pointers[num_left+1:]
                
                above_node = BPTreeNode(leaf=False, root=True)
                above_node.keys = [self.keys[num_left]]
                above_node.pointers = [self, right_node]
                
                self.keys = self.keys[:num_left]
                self.pointers = self.pointers[:num_left+1]
                
                return above_node
            return self
        
    def search(self, key):
        """
        Finds and returns the list of values corresponding to the key
        If not found, return None
        """
        if self.leaf:
            for i in range(len(self.keys)):
                if key == self.keys[i]:
                    return self.pointers[i]
            return None
        else:
            for i in range(len(self.keys)):
                if key < self.keys[i]:
                    return self.pointers[i].search(key)
            return self.pointers[-1].search(key)
    
    def search_first_gte(self, key):
        """
        A utility function used by search_range to return the first leaf node >= key
        If found, return the leaf node containing the key and the index of the key in the node
        If not found, i.e. key is smaller than all keys, return None
        """
        if self.leaf:
            for i in range(len(self.keys)):
                if self.keys[i] >= key:
                    return self, i
            if self.pointers[-1] == None:
                # this is true if self is the rightmost leaf node
                return None
            # if leaf node is not rightmost, we know the first key of the immediate right neightbour will satisfy condition
            # because self.pointers[-1].keys[0] >= some LB > key
            return self.pointers[-1], 0
        else:
            # find the subtree to recursively call on
            for i in range(len(self.keys)):
                if key < self.keys[i]:
                    return self.pointers[i].search_first_gte(key)
            return self.pointers[-1].search_first_gte(key)
        
    
    def search_range(self, lower, upper):
        """
        Returns a list of all values whose keys are in the range [lower, upper] inclusive
        If lower is None, it is treated as no lower bound
        If upper is None, it is trated as no upper bound
        If both are None, return all values
        """
        if lower == None:
            lower = float("-inf")
        if upper == None:
            upper = float("inf")
        if lower > upper:
            return []
        first_gte = self.search_first_gte(lower)
        
        res = []
        if first_gte == None:
            return res
        node, pos = first_gte
        while node:
            for i in range(pos, len(node.keys)):
                if node.keys[i] > upper:
                    # current and all other leaf nodes on the road are greater than upper bound and not part of res
                    # so we can just return res
                    return res
                res.append(node.pointers[i])
            # move to the immediate right neighbour
            node = node.pointers[-1]
            pos = 0
        # this return is needed if the res includes the rightmost leaf node
        return res
    
    def find_min(self):
        """
        Utility function to minimum value rooted at self
        # DEPRECATED (nodes should maintain self.min_val on the fly) - DO NOT USE EXCEPT FOR DEBUGGING
        """
        cur = self
        while type(cur.pointers[0]) == BPTreeNode:
            cur = cur.pointers[0]
        return cur.keys[0]
    
    def set_min_as_first_key(self):
        self.min_val = self.keys[0]
    
    def delete(self, key, left_neighbour, right_neighbour, parent, upper_bound, lower_bound):
        """
        different return values have different meaning
        if return [4] => key doesn't exist
        if return [0] => the function was called on a leaf node, and there is no merging done
        if return [1, 0] => the function was called on a leaf node and that leaf node merged with its right sibling
        if return [1, 1] => the function was called on a leaf node and that leaf node merged with its left sibling
        if return [2] => the function was called on a non-leaf node and there was no merging done
        if return [3, 0] => the function was called on a non-leaf node and that non-leaf node merged with its right sibling
        if return [3, 1] => the function was called on a non-leaf node and that non-leaf node merged with its left sibling
        """
        # BASE CASE
        if self.leaf:
            exists = False
            for i in range(len(self.keys)):
                if self.keys[i] == key:
                    exists = True
                    self.keys.pop(i)
                    self.pointers.pop(i)
                    break
            if not exists:
                return [4]
            # at this point, the pointer and key in leaf node is removed
            # check to see if min num keys for leaf is satisfied
            if len(self.keys) >= self.min_leaf_keys:
                return [0]
            # we know min leaf key is not satisfied but its ok for root
            if parent == None:
                assert self.root == True
                return [0]
            # check if can take from left neighbour
            if left_neighbour and len(left_neighbour.keys) > self.min_leaf_keys:
                self.keys = [left_neighbour.keys.pop()] + self.keys
                self.pointers = [left_neighbour.pointers.pop(-2)] + self.pointers
                return [0]
            # check if can take from right neighbour
            if right_neighbour and len(right_neighbour.keys) > self.min_leaf_keys:
                self.keys += [right_neighbour.keys.pop(0)]
                self.pointers = self.pointers[:-1] + [right_neighbour.pointers.pop(0)] + [self.pointers[-1]]
                return [0]
            # if neither works, merge
            # try merge right
            if right_neighbour:
                assert len(self.keys) + len(right_neighbour.keys) <= self.max_keys
                self.pointers.pop()
                self.keys += right_neighbour.keys
                self.pointers += right_neighbour.pointers
                return [1, 0]
            # try merge left
            elif left_neighbour:
                assert len(self.keys) + len(left_neighbour.keys) <= self.max_keys
                self.keys = left_neighbour.keys + self.keys
                self.pointers = left_neighbour.pointers[:-1] + self.pointers
                return [1, 1]
            else:
                raise Exception("Could neither borrow nor merge")
        
        # RECURSIVE
        else:
            pos = None
            for i in range(len(self.keys)):
                if key < self.keys[i]:
                    pos = i
                    break
            if pos == None:
                pos = len(self.keys)
            # pointers[pos] is the node we recursively call on
            # get the left and right sibling of the node we recursively call on so its easier to do merging/borrowing
            call_left_neighbour = self.pointers[pos-1] if pos-1 >= 0 else None
            call_right_neighbour = self.pointers[pos+1] if pos+1 < len(self.pointers) else None
            # get lower/upper bound which is needed for merging
            if 0 <= pos < len(self.keys):
                call_upper_bound = self.keys[pos]
            else:
                call_upper_bound = upper_bound
            if 0 <= pos - 1 < len(self.keys):
                call_lower_bound = self.keys[pos-1]
            else:
                call_lower_bound = lower_bound
            # res is the result from the recursive call
            res = self.pointers[pos].delete(key,
                                            call_left_neighbour,
                                            call_right_neighbour,
                                            self, call_upper_bound,
                                            call_lower_bound)
            if res[0] == 4:
                # key not found
                return [4]
            if res[0] == 0:
                # res == 0 means that there was no merging from leaves
                # simply need to update the value of keys[pos-1] and keys[pos]
                if pos < len(self.keys):
                    self.keys[pos] = self.pointers[pos+1].keys[0]
                if pos-1 >= 0:
                    self.keys[pos-1] = self.pointers[pos].keys[0]
                return [2]
            elif res[0] == 2:
                for ppos in range(pos-1, pos+2):
                    if 1 <= ppos < len(self.pointers):
                        self.keys[ppos-1] = self.pointers[ppos].find_min() # min_val
                return [2]
            else:
                if res[1] == 0:
                    # below merged right
                    # delete self.pointers[pos+1] and self.keys[pos]
                    self.pointers.pop(pos+1)
                    self.keys.pop(pos)
                elif res[1] == 1:
                    # below merged left
                    # delete self.pointers[pos-1] and self.keys[pos-1]
                    self.pointers.pop(pos-1)
                    self.keys.pop(pos-1)
                for ppos in range(pos-1, pos+2):
                    if 1 <= ppos < len(self.pointers):
                        self.keys[ppos-1] = self.pointers[ppos].find_min()
                # check if non leaf min keys is satisfied
                if len(self.keys) >= self.min_non_leaf_keys or parent == None:
                    return [2]
                # borrow from left
                if left_neighbour and len(left_neighbour.keys) > self.min_non_leaf_keys:
                    self.keys = [left_neighbour.keys.pop()] + self.keys
                    self.pointers = [left_neighbour.pointers.pop()] + self.pointers
                    self.keys[0] = self.pointers[1].keys[0]
                    return [2]
                # borrow from right
                if right_neighbour and len(right_neighbour.keys) > self.min_non_leaf_keys:
                    self.keys += [right_neighbour.keys.pop(0)]
                    self.pointers += [right_neighbour.pointers.pop(0)]
                    self.keys[-1] = self.pointers[-1].keys[0]
                    return [2]
                # if neither works, merge
                if right_neighbour:
                    assert upper_bound != float("inf")
                    x = len(self.keys)
                    self.keys += [upper_bound] + right_neighbour.keys
                    self.pointers += right_neighbour.pointers
                    self.keys[x] = self.pointers[x+1].find_min()
                    return [3, 0]
                elif left_neighbour:
                    assert lower_bound != float("-inf")
                    self.keys = left_neighbour.keys + [lower_bound] + self.keys
                    self.pointers = left_neighbour.pointers + self.pointers
                    self.keys[len(left_neighbour.keys)] = self.pointers[len(left_neighbour.keys)+1].find_min()
                    return [3, 1]
                else:
                    raise Exception("FUCKED")
    
    def validate(self):
        # assume that the tree has >1 node
        # check number of keys and pointers are valid
        if self.root:
            pass
        elif self.leaf:
            assert self.min_leaf_keys <= len(self.keys) <= self.max_keys
        else:
            assert self.min_non_leaf_keys <= len(self.keys) <= self.max_keys
        assert len(self.pointers) - len(self.keys) == 1
        # check that values in a node are sorted
        for i in range(len(self.keys) - 1):
            assert self.keys[i] <= self.keys[i+1]
        if type(self.pointers[0]) != BPTreeNode:
            # already reach leaf nodes
            return self.keys[0]
        # check that values in a level are sorted
        for i in range(len(self.pointers)-1):
            assert self.pointers[i].keys[0] <= self.pointers[i+1].keys[0]
        # recursively check
        for i in range(len(self.pointers)):
            if self.pointers[i] == None:
                continue
            min_val = self.pointers[i].validate()
            if i > 0:
                assert self.keys[i-1] == min_val
        return self.pointers[0].validate()

In [75]:
tree = BPTree()

In [60]:
mock_data = {x: f"{x}x" for x in [5,20,35,1,2,5,6,9,10,18,19,11,12,22,24,25,31,32,34,35,37,38]}

In [61]:
for k, v in mock_data.items():
    tree.insert(k, v)
    tree.validate()
for k, v in mock_data.items():
    assert tree.search(k) == [v]

In [67]:
tree.delete(4)

In [68]:
tree.show()

6 19 35 |

1 2 5 | 6 9 18 | 19 20 31 32 | 35 37 38 |



In [62]:
tree.show()
for v in [25,24,12,11,10,22,34]:
    tree.delete(v)
    print(f"deleted: {v}\n")
    tree.show()
    tree.validate()
    

9 19 25 35 |

1 2 5 6 | 9 10 11 12 18 | 19 20 22 24 | 25 31 32 34 | 35 37 38 |

deleted: 25

9 19 31 35 |

1 2 5 6 | 9 10 11 12 18 | 19 20 22 24 | 31 32 34 | 35 37 38 |

deleted: 24

9 19 31 35 |

1 2 5 6 | 9 10 11 12 18 | 19 20 22 | 31 32 34 | 35 37 38 |

deleted: 12

9 19 31 35 |

1 2 5 6 | 9 10 11 18 | 19 20 22 | 31 32 34 | 35 37 38 |

deleted: 11

9 19 31 35 |

1 2 5 6 | 9 10 18 | 19 20 22 | 31 32 34 | 35 37 38 |

deleted: 10

6 19 31 35 |

1 2 5 | 6 9 18 | 19 20 22 | 31 32 34 | 35 37 38 |

deleted: 22

6 19 35 |

1 2 5 | 6 9 18 | 19 20 31 32 34 | 35 37 38 |

deleted: 34

6 19 35 |

1 2 5 | 6 9 18 | 19 20 31 32 | 35 37 38 |



In [76]:
import random
num_trials = 10000
# for t in range(num_trials):
while True:
    tree = BPTree()
    test_data = list(range(0, 100))
    
    random.shuffle(test_data)
    for x in test_data:
        tree.insert(x, "")
        tree.validate()
    random.shuffle(test_data)
    for x in test_data:
        
        tree.delete(x)
        print("del", x)
        tree.show()
        tree.validate()

del 22
16 34 63 85 |

5 9 13 | 20 24 28 | 39 45 49 53 56 60 | 68 74 78 81 | 88 91 97 |

0 1 2 3 4 | 5 6 7 8 | 9 10 11 12 | 13 14 15 | 16 17 18 19 | 20 21 23 | 24 25 26 27 | 28 29 30 31 32 33 | 34 35 36 37 38 | 39 40 41 42 43 44 | 45 46 47 48 | 49 50 51 52 | 53 54 55 | 56 57 58 59 | 60 61 62 | 63 64 65 66 67 | 68 69 70 71 72 73 | 74 75 76 77 | 78 79 80 | 81 82 83 84 | 85 86 87 | 88 89 90 | 91 92 93 94 95 96 | 97 98 99 |

del 21
16 34 63 85 |

5 9 13 | 19 24 28 | 39 45 49 53 56 60 | 68 74 78 81 | 88 91 97 |

0 1 2 3 4 | 5 6 7 8 | 9 10 11 12 | 13 14 15 | 16 17 18 | 19 20 23 | 24 25 26 27 | 28 29 30 31 32 33 | 34 35 36 37 38 | 39 40 41 42 43 44 | 45 46 47 48 | 49 50 51 52 | 53 54 55 | 56 57 58 59 | 60 61 62 | 63 64 65 66 67 | 68 69 70 71 72 73 | 74 75 76 77 | 78 79 80 | 81 82 83 84 | 85 86 87 | 88 89 90 | 91 92 93 94 95 96 | 97 98 99 |

del 32
16 34 63 85 |

5 9 13 | 19 24 28 | 39 45 49 53 56 60 | 68 74 78 81 | 88 91 97 |

0 1 2 3 4 | 5 6 7 8 | 9 10 11 12 | 13 14 15 | 16 17 18 | 19 20 23 |

AssertionError: 

In [31]:
data = parse_data()

In [32]:
data[0][1]

5.6

In [35]:
tree = BPTree()
for i, record in enumerate(data):
    if i % 1000 == 0:
        print(i)
    tree.insert(record[1], record)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [ ]:
tree.search(5.6)

In [ ]:
tree.delete(5.6)

In [ ]:
tree.search(5.6)